<a href="https://colab.research.google.com/github/eklz/Prediction-de-likes-Instagram/blob/master/InstaScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1,372 kB]
Get:15 http:

In [0]:
from selenium import webdriver
import datetime as dt
from bs4 import BeautifulSoup as bs
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import requests
import json
import os

In [13]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/InstaScrap/'
if not os.path.exists(root_path):
      os.makedirs(root_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class User():

  def __init__(self, username):

    self.username=username
    self.URL='http://www.instagram.com/'+username+'/?hl=fr'
    self.data=self.loadData()
    self.nbPubli= self.data['edge_owner_to_timeline_media']['count']
    self.followed_by=self.data['edge_followed_by']['count']
    self.follow=self.data['edge_follow']['count']
    self.business=self.data['business_category_name']

  def loadData(self):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    browser = webdriver.Chrome('chromedriver',options=chrome_options)
    browser.get(self.URL)
    source = browser.page_source
    browser.quit()

    data = bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
    page_json = script.text.split(' = ', 1)[1].rstrip(';')
    data = json.loads(page_json)
    data = data['entry_data']['ProfilePage'][0]['graphql']['user']
    return data

  def loadPictures(self):
    i=0
    for publi in self.data['edge_owner_to_timeline_media']['edges']:
      publi=publi['node']

      if publi['__typename'] == 'GraphImage':
        i+=1

        link='https://www.instagram.com'+'/p/'+publi['shortcode']+'/'
        analyse=publi['accessibility_caption']
        comments=publi['edge_media_to_comment']['count']
        likes=publi['edge_liked_by']['count']
        try:
          location_id=publi['location']['id']
          location_name=publi['location']['name']
        except:
          location_id=None
          location_name=None
        timestamp=publi['taken_at_timestamp']
        time=dt.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        r = requests.get(publi['display_url'])

        with open(self.userDir + '/'+str(i) + '.jpg', 'wb') as f:
          f.write(r.content)

        carac={'user':self.username, 'likes':likes, 'comments':comments, 
               'location_id':location_id, 'location_name':location_name, 
               'time':time , 'analyse':analyse}
        with open(self.userDir + '/' + str(i) + '.txt', 'w') as file:
          json.dump(carac,file)

  def createUser(self):

    self.userDir = os.path.join(root_path, self.username)
    if not os.path.exists(self.userDir):
      os.makedirs(self.userDir)

    dic={'name':self.username, 'nbPubli': self.nbPubli, 
        'nbFollowers':self.followed_by, 'nbFollow':self.follow, 
        'typeBusiness':self.business}

    with open(self.userDir + '/' + self.username + '.txt', 'w') as file:
        json.dump(dic,file)




In [0]:
with open(root_path+'users.txt','r') as f:
  text=f.readline()
listUsers=text.split('\t')

In [0]:
for user in listUsers:
  infl=User(user)
  infl.createUser()
  infl.loadPictures()




In [18]:
print(infl.data)

{'biography': '@euphoria on @hbo Sundays @ 10pm', 'blocked_by_viewer': False, 'restricted_by_viewer': None, 'country_block': False, 'external_url': None, 'external_url_linkshimmed': None, 'edge_followed_by': {'count': 67987451}, 'followed_by_viewer': False, 'edge_follow': {'count': 1448}, 'follows_viewer': False, 'full_name': 'Zendaya', 'has_ar_effects': False, 'has_channel': False, 'has_blocked_viewer': False, 'highlight_reel_count': 3, 'has_requested_viewer': False, 'id': '9777455', 'is_business_account': False, 'is_joined_recently': False, 'business_category_name': None, 'category_id': None, 'overall_category_name': None, 'is_private': False, 'is_verified': True, 'edge_mutual_followed_by': {'count': 0, 'edges': []}, 'profile_pic_url': 'https://scontent-hkt1-1.cdninstagram.com/v/t51.2885-19/s150x150/43146007_338219383412225_7074956904937553920_n.jpg?_nc_ht=scontent-hkt1-1.cdninstagram.com&_nc_ohc=cDZWZIw6DFEAX-okfkh&oh=14b8f17e105c12b68b8b7d42c8bf5775&oe=5ECB7FC5', 'profile_pic_url_h